## コーパスの名前、電話番号、メールアドレスをマスキングする
- ただし名前は、後ろに敬称（さん、くん、君、様）が付くものに限る

In [ ]:
!pip install -U hojichar==0.9.0
!pip show hojichar
!pip install regex


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.2 MB/s eta 0:00:00
Name: hojichar
Version: 0.9.0
Summary: Text preprocessing management system.
Home-page: https://github.com/HojiChar/HojiChar
Author: kenta.shinzato
Author-email: hoppiece@gmail.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: mmh3, numpy, tqdm
Required-by: 


In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.5 MB/s eta 0:00:00


In [ ]:
import regex as re
import json
import random
import hojichar
from multiprocessing import Pool, cpu_count
from faker import Faker

fake = Faker('ja_JP')  # 日本語のダミーデータを生成するためのFakerインスタンス

num_samples = 100000
dummy_texts = [
    "こんにちは田中君、これはテストデータです。電話番号は08098765432です。佐藤さんのメールアドレスはdef@example.comです。",
    "山田くんのメールアドレスはjtwmdagp@gmail.comで、電話番号は03-4499-3322です。",
    "HojiCharを使用して前処理を行います。村上様の電話番号は、075-4329-9899です。",
]

def replace_entities(text):
    # 名前を置換
    text = re.sub(r'(?P<name>\p{Script=Han}+)(?P<honorific>さん|くん|様|君)',
                  lambda m: fake.last_name() + m.group('honorific'), text)

    # 電話番号を置換
    text = re.sub(r'\d{2,4}-?\d{2,4}-?\d{2,4}', lambda x: fake.phone_number(), text)

    # メールアドレスを置換
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', fake.email(), text)

    return text

def process_text(text):
    replaced_text = replace_entities(text)
    # HojiCharの処理
    cleaner = hojichar.Compose([
        hojichar.document_filters.DocumentNormalizer(),
        hojichar.document_filters.JSONDumper(),
    ])
    document = hojichar.Document(json.dumps({"text": replaced_text}))
    cleaned_text = cleaner.apply(document).text
    return cleaned_text

def main():
    random_texts = [random.choice(dummy_texts) for _ in range(num_samples)]
    num_cores = cpu_count()

    with Pool(num_cores) as pool:
        processed_texts = pool.map(process_text, random_texts)

    with open("your_text.jsonl", "w") as f:
        for processed_text in processed_texts:
            f.write(processed_text + "\n")

    # 結果の表示
    num_display = 100
    with open("your_text.jsonl", "r") as f:
        for i, line in enumerate(f):
            if i >= num_display:
                break
            data = json.loads(line)
            actual_text = data['text']
            print(actual_text)

if __name__ == "__main__":
    main()

{'text': 'こんにちは伊藤君、これはテストデータです。電話番号は090-4744-8002です。伊藤さんのメールアドレスはosamutakahashi@example.orgです。'}
{'text': 'こんにちは山下君、これはテストデータです。電話番号は090-6792-1281です。三浦さんのメールアドレスはmiki72@example.netです。'}
{'text': 'HojiCharを使用して前処理を行います。斉藤様の電話番号は、090-9898-6232です。'}
{'text': '村上くんのメールアドレスはkhasegawa@example.orgで、電話番号は080-2584-9901です。'}
{'text': '加藤くんのメールアドレスはsotaro39@example.netで、電話番号は080-6676-2629です。'}
{'text': '中村くんのメールアドレスはshoheisuzuki@example.comで、電話番号は090-8759-0351です。'}
{'text': '斎藤くんのメールアドレスはasukanakamura@example.netで、電話番号は080-9513-1672です。'}
{'text': '伊藤くんのメールアドレスはwatanabetaichi@example.netで、電話番号は50-9740-3576です。'}
{'text': 'HojiCharを使用して前処理を行います。石川様の電話番号は、57-2630-2826です。'}
{'text': 'HojiCharを使用して前処理を行います。山本様の電話番号は、080-4575-5997です。'}
{'text': '石川くんのメールアドレスはwnakamura@example.orgで、電話番号は070-1662-8880です。'}
{'text': 'HojiCharを使用して前処理を行います。鈴木様の電話番号は、60-9498-1872です。'}
{'text': 'HojiCharを使用して前処理を行います。伊藤様の電話番号は、070-4728-5466です。'}
{'text': '小林くんのメールアドレスはishiiyoko@example.comで、電話番号は090-3845-9222です。'}
{'text': '山